In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_llm_cache import ElasticsearchLLMCache, ElasticsearchLLMFilter
from elasticsearch.exceptions import NotFoundError

# common libraries
from dotenv import load_dotenv
import os
from os import environ
import openai
from icecream import ic

# load our environment file
load_dotenv()

es_url = f"https://{os.environ['elasticsearch_user']}:{os.environ['elasticsearch_pw']}@{os.environ['elasticsearch_host']}:{os.environ['elasticsearch_port']}"
# es_index= os.environ['elasticsearch_index']
os.environ['OPENAI_API_KEY'] = os.environ['openai_api_key']
open_api_key=os.environ['OPENAI_API_KEY']

# define our API Key
openai.api_key = os.getenv("openai_api_key")


In [2]:
es = Elasticsearch([es_url])

cache_index_name = 'llm_cache_test'
filter_index_name = 'llm_filter_test'
model_id = 'sentence-transformers__msmarco-minilm-l-12-v3'

# if es.ml.get_trained_models(model_id=model_id):
#     ic(f'{model_id} exists.')
# else:
#     ic(f'{model_id} does not exist.')

# if es.indices.exists(index=cache_index_name):
#     ic(f'{cache_index_name} exists.')
# else:
#     ic(f'{cache_index_name} does not exist.')
#     es.indices.create(index=cache_index_name)

# if es.indices.exists(index=filter_index_name):
#     ic(f'{filter_index_name} exists.')
# else:
#     ic(f'{filter_index_name} does not exist.')
#     es.indices.create(index=filter_index_name)

In [3]:
import os
import time
#print(os.environ['ELASTIC_CLOUD_ID'])
#time.sleep(10)
from elasticsearch import Elasticsearch

from elasticsearch_llm_cache import (
    ElasticsearchLLMCache,  
    ElasticsearchLLMFilter,
)

from pprint import pprint
import time

es_client = es

if es_client.indices.exists(index=cache_index_name):
    es_client.indices.delete(index=cache_index_name)
    ic(f'{cache_index_name} exists, deleting.')

if es_client.indices.exists(index=filter_index_name):
    es_client.indices.delete(index=filter_index_name)
    ic(f'{filter_index_name} exists, deleting.')

# es_client.indices.create(index=cache_index_name)
# es_client.indices.create(index=filter_index_name)


# Initialize your caching class
cache = ElasticsearchLLMCache(es_client=es_client, index_name=cache_index_name, es_model_id=model_id, create_index=False)
cache.create_index(dims=768)

filter = ElasticsearchLLMFilter(es_client=es_client, index_name=filter_index_name, es_model_id=model_id, create_index=False)
filter.create_index(dims=768)


INFO:elastic_transport.transport:HEAD https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_cache_test [status:200 duration:0.395s]
INFO:elastic_transport.transport:DELETE https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_cache_test [status:200 duration:0.087s]
ic| f'{cache_index_name} exists, deleting.': 'llm_cache_test exists, deleting.'
INFO:elastic_transport.transport:HEAD https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_filter_test [status:200 duration:0.032s]
INFO:elastic_transport.transport:DELETE https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_filter_test [status:200 duration:0.087s]
ic| f'{filter_index_name} exists, deleting.': 'llm_filter_test exists, deleting.'
INFO:elastic_transport.transport:HEAD https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_cache_test [status:404 duration:0.032s]
INFO:elastic_transport.transport:PUT https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_cache_test [status:200 duration:0.137s]
ic| f

{'cache_index': 'llm_filter_test', 'created_new': True}

In [5]:
# filter.clear()
subjects = ['sports', 'money', 'celebrities', 'basketball', 'soccer', 'tennis', 'skiing', 'animals', 'snakes']
filter_contents = []

for subject in subjects:
    filter.add(prompt=subject, response=f"Sorry, I can't talk about {subject}.")

# for subject in subjects:
#     filter_contents.append({'prompt': subject, 'response': f"Sorry, I can't talk about {subject}."})


filter_results = filter.add_bulk(documents=filter_contents)
filter.list()


INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/_ml/trained_models/sentence-transformers__msmarco-minilm-l-12-v3/_infer [status:200 duration:0.041s]
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_filter_test/_search [status:200 duration:0.033s]
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/_ml/trained_models/sentence-transformers__msmarco-minilm-l-12-v3/_infer [status:200 duration:0.035s]
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_filter_test/_doc [status:400 duration:0.035s]
ERROR:elasticsearch_llm_cache:BadRequestError(400, 'document_parsing_exception', "[1:7896] failed to parse: The [dense_vector] field [prompt_vector] in doc [document with id 'd2-5yYsBhjjTaImgxtNM'] has a different number of dimensions [384] than defined in the mapping [768]")
INFO:elastic_transport.transport:POST http

{}

In [32]:
results = filter.query(prompt_text="How do I code a linked list with python?", similarity_threshold=0.2, size=1)
ic(results)


ic| prompt_text: 'How do I code a linked list with python?'
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/_ml/trained_models/sentence-transformers__msmarco-minilm-l-12-v3/_infer [status:200 duration:0.069s]
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_filter_test/_search [status:200 duration:0.034s]
ic| hit['_score']: 1.1454514
    hit['_source']['prompt']: 'snakes'
    score: 0.1454514
    threshold: 0.2
ic| results: False


False

In [ ]:
filter.query('I will vote for a democratic politician', similarity_threshold=0.25
ic(results)

In [ ]:
prompt = "Hello, my name is Morgan?"
llm_response = "I'm here to assist you!"  # Assume this response is fetched from LLM

cache_contents = [
    {"prompt": "Hello, my name is Morgan", "response": "Hi Morgan"},
    {"prompt": "Hello, my name is Adam", "response": "Hi Adam"},
    {"prompt": "Hello, my name is David", "response": "Hi David"},
    {"prompt": "Hello, my name is Morgan", "response": "Hi Morgan"},
]

cache_results = cache.add_bulk(documents=cache_contents)

for item in cache_contents:
    llm_response = cache.query(prompt_text=item['prompt'], )
    ic(llm_response)



In [ ]:
!eland_import_hub_model --url "$es_url" \
      --hub-model-id "sentence-transformers/msmarco-MiniLM-L-12-v3" \
      --task-type "text_embedding"